In [29]:
import pandas as pd
import geopandas as gpd

In [7]:
#read files
energy = pd.read_csv("electricity-renewables.csv")
solar = pd.read_csv("solar-energy-consumption.csv")
wind = pd.read_csv("wind-generation.csv")
hydropower = pd.read_csv("hydropower-consumption.csv")

In [9]:
#filter for 2023
energy2023 = energy[energy["Year"] == 2023]
solar2023 = solar[solar["Year"] == 2023]
wind2023 = wind[wind["Year"] == 2023]
hydropower2023 = hydropower[hydropower["Year"] == 2023]

In [19]:
#filter necessary columns from each dataset
solar_filtered = solar2023[['Entity', 'Electricity from solar - TWh']]
hydro_filtered = hydropower2023[['Entity', 'Electricity from hydro - TWh']]
wind_filtered = wind2023[['Entity', 'Electricity from wind - TWh']]

#merge energy2023 with solar, hydro, and wind datasets
energy2023_complete = (
    energy2023
    .merge(solar_filtered, on='Entity', how='inner')
    .merge(hydro_filtered, on='Entity', how='inner')
    .merge(wind_filtered, on='Entity', how='inner')
)

In [23]:
#calculate energy for others
energy2023_complete["Electricity from others - TWh"] = (
    energy2023_complete["Electricity from renewables - TWh"] 
    - energy2023_complete["Electricity from solar - TWh"] 
    - energy2023_complete["Electricity from hydro - TWh"] 
    - energy2023_complete["Electricity from wind - TWh"]
)

In [25]:
#calculate the percentage contribution for each energy source
energy2023_complete["% from solar"] = (
    energy2023_complete["Electricity from solar - TWh"] 
    / energy2023_complete["Electricity from renewables - TWh"]
) * 100

energy2023_complete["% from hydro"] = (
    energy2023_complete["Electricity from hydro - TWh"] 
    / energy2023_complete["Electricity from renewables - TWh"]
) * 100

energy2023_complete["% from wind"] = (
    energy2023_complete["Electricity from wind - TWh"] 
    / energy2023_complete["Electricity from renewables - TWh"]
) * 100

energy2023_complete["% from others"] = (
    energy2023_complete["Electricity from others - TWh"] 
    / energy2023_complete["Electricity from renewables - TWh"]
) * 100

In [27]:
energy2023_complete.head()

,Entity,Code,Year,Electricity from renewables - TWh,Electricity from solar - TWh,Electricity from hydro - TWh,Electricity from wind - TWh,Electricity from others - TWh,% from solar,% from hydro,% from wind,% from others
0,ASEAN (Ember),NaN,2023,316.21000,38.510000,192.33000,16.11000,6.926000e+01,12.178615,60.823503,5.094716,21.903166
1,Africa (EI),NaN,2023,218.36761,19.197226,161.64522,27.82143,9.703734e+00,8.791242,74.024357,12.740640,4.443761
2,Africa (Ember),NaN,2023,213.96000,27.140000,150.80000,26.99000,9.030000e+00,12.684614,70.480464,12.614507,4.220415
3,Algeria,DZA,2023,0.73378,0.649000,0.06878,0.01600,-3.000000e-08,88.446132,9.373382,2.180490,-0.000004
4,Argentina,ARG,2023,53.24000,3.250000,33.17000,14.49000,2.330000e+00,6.104433,62.302780,27.216379,4.376409


In [59]:
#save the csv
energy2023_complete.to_csv("energy2023_complete.csv")

In [31]:
#read in the shapefile
worldmap = gpd.read_file("WB_countries_Admin0_10m.shp")

In [65]:
#perform the join based on ISO_A3 and Code
world_energy = worldmap.merge(energy2023_complete, 
                                left_on="ISO_A3", 
                                right_on="Code", 
                                how="left")

In [67]:
#filter for South America
world_energy_south_america = world_energy[world_energy["CONTINENT"] == "South America"]

In [71]:
#save the filtered data as a GeoPackage
world_energy_south_america.to_file("world_energy_south_america.gpkg", driver="GPKG")